In [ ]:
from typing import TypedDict, List
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END
import random
import os

# ============================
# 1️⃣ 상태 정의
# ============================
class CafeMessageState(TypedDict):
    messages: List
    inquiry_type: str

# ============================
# 2️⃣ LLM & Vector DB 로드
# ============================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Vector DB 로드 (이미 ../db/cafe_db에 생성되어 있음)
db_path = "../db/cafe_db"
if os.path.exists(db_path):
    menu_db = FAISS.load_local(
        db_path, 
        embeddings, 
        allow_dangerous_deserialization=True  # ✅ 추가
    )
else:
    raise FileNotFoundError(f"{db_path} 에 Vector DB 파일이 없습니다.")

# ============================
# 3️⃣ 핸들러 정의
# ============================

# 질문 분류
def classify_inquiry(state: CafeMessageState):
    user_msg = state["messages"][-1].content
    inquiry_type = "unknown"
    if "추천" in user_msg or "추천 메뉴" in user_msg:
        inquiry_type = "recommend"
    elif "가격" in user_msg:
        inquiry_type = "price"
    elif "메뉴" in user_msg or "전체" in user_msg:
        inquiry_type = "menu_list"
    return {**state, "inquiry_type": inquiry_type}

# 추천/가격 → 하나만 선택
def handle_single_menu(state: CafeMessageState):
    docs = menu_db.similarity_search("추천", k=1)  # 가장 관련성 높은 1개 메뉴
    if docs:
        selected = docs[0].page_content
    else:
        selected = "죄송합니다. 추천할 메뉴를 찾을 수 없습니다."
    return {
        **state,
        "messages": state["messages"] + [AIMessage(content=f"추천 메뉴 정보:\n{selected}")]
    }

# 전체 메뉴 → 전부 출력 (포맷 유지)
def handle_menu_list(state: CafeMessageState):
    docs = menu_db.similarity_search("메뉴", k=20)  # 전체 메뉴
    menu_texts = []
    for i, doc in enumerate(docs, start=1):
        menu_texts.append(f"{i}. {doc.page_content}")
    return {
        **state,
        "messages": state["messages"] + [AIMessage(content="저희 카페 전체 메뉴:\n" + "\n\n".join(menu_texts))]
    }

# 알 수 없는 질문
def handle_unknown(state: CafeMessageState):
    return {
        **state,
        "messages": state["messages"] + [AIMessage(content="죄송합니다. 메뉴, 가격, 추천 관련 질문을 해주세요.")]
    }

# ============================
# 4️⃣ 그래프 정의
# ============================
graph = StateGraph(CafeMessageState)

graph.add_node("classify", classify_inquiry)
graph.add_node("single_menu", handle_single_menu)
graph.add_node("menu_list", handle_menu_list)
graph.add_node("unknown", handle_unknown)

graph.set_entry_point("classify")

graph.add_conditional_edges(
    "classify",
    lambda s: s["inquiry_type"],
    {
        "recommend": "single_menu",
        "price": "single_menu",
        "menu_list": "menu_list",
        "unknown": "unknown"
    }
)

graph.add_edge("single_menu", END)
graph.add_edge("menu_list", END)
graph.add_edge("unknown", END)

app = graph.compile()

# ============================
# 5️⃣ 테스트
# ============================
state1 = CafeMessageState(messages=[HumanMessage(content="추천 메뉴와 가격을 알려줘")], inquiry_type="")
result1 = app.invoke(state1)
print("\n--- 답변1 ---")
print(result1["messages"][-1].content)

state2 = CafeMessageState(messages=[HumanMessage(content="전체 메뉴 알려줘")], inquiry_type="")
result2 = app.invoke(state2)
print("\n--- 답변2 ---")
print(result2["messages"][-1].content)
